# Migrating TensorFlow EfficientNet to Habana Gaudi<sup>TM</sup>

In this Jupyter notebook, we will learn how to migrate EfficientNet in public TensorFlow [model garden](https://github.com/tensorflow/models/tree/master/official/vision/image_classification) to Habana Gaudi<sup>TM</sup> device with very limited code changes. We will first modify an existing YAML configuration file for GPU and enable the model training on CPU. Then we will add code to the training script to load Habana software modules and enable it on HPU.

## Check current directory

In [ ]:
%pwd

## Clone TensorFlow [models](https://github.com/tensorflow/models.git)  repository to the current directory

In [ ]:
!git clone https://github.com/tensorflow/models.git

## Clone Habana [Model-References](https://github.com/HabanaAI/Model-References.git) repository branch 0.15.4 to the current directory

In [ ]:
!git clone -b 0.15.4 https://github.com/HabanaAI/Model-References.git

## Check the repositories were cloned successfully

In [ ]:
%ls

## Check current PYTHONPATH

In [ ]:
%env PYTHONPATH

## Add TensorFlow Models repository location and Habana Model-References repository location to the PYTHONPATH

In [ ]:
%set_env PYTHONPATH=/root/examples/Model-References:/root/examples/models:/usr/lib/habanalabs/:/root

## Check the repository locations are added to the PYTHONPATH

In [ ]:
%env PYTHONPATH

## We will use Keras EfficientNet at https://github.com/tensorflow/models/tree/master/official/vision/image_classification as the model migration example. First of all, we will enable the model training on CPU.

In [ ]:
%cd models/official/vision/image_classification

In [ ]:
%ls

## Enable EfficientNet training on CPU with synthetic data by modifying the following configuration file for GPU:

[configs/examples/efficientnet/imagenet/efficientnet-b0-gpu.yaml](../edit/models/official/vision/image_classification/configs/examples/efficientnet/imagenet/efficientnet-b0-gpu.yaml)
   * Line 6:  change distribution_strategy to `off`
   * Line 7:  change num_gpus to `0`
   * Line 11: change builder to `synthetic`
   * Line 23: change builder to `synthetic`
   * Line 50: insert `steps: 1000`
   * Line 51: change epochs to `1`
   * Line 53: insert `skip_eval: True`
   
   Save the file.


## Run the following command to launch the training on CPU for 1000 iterations. Check the performance from output.

In [ ]:
!python3 classifier_trainer.py --mode=train_and_eval --model_type=efficientnet --dataset=imagenet --model_dir=$HOME/log_cpu --data_dir=$HOME --config_file=configs/examples/efficientnet/imagenet/efficientnet-b0-gpu.yaml

## Enable EfficientNet training on HPU with synthetic data by modifying  `classifier_trainer.py`:

[classifier_trainer.py](../edit/models/official/vision/image_classification/classifier_trainer.py)
   * Line 443:  Insert the following 3 lines of code:
   ```
   from TensorFlow.common.library_loader import load_habana_module
log_info_devices = load_habana_module()
logging.info('Devices:\n%s', log_info_devices)
   ```
   
   Save the file.

## Launch the training on HPU and check the performance in the output.

In [ ]:
!python3 classifier_trainer.py --mode=train_and_eval --model_type=efficientnet --dataset=imagenet --model_dir=$HOME/log_hpu --data_dir=$HOME --config_file=configs/examples/efficientnet/imagenet/efficientnet-b0-gpu.yaml